In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip3 install dask_ml
!pip3 install langdetect
import dask.dataframe as dd
from dask_ml.feature_extraction.text import CountVectorizer
from dask_ml.preprocessing import StandardScaler
from dask_ml.model_selection import train_test_split
from dask_ml.model_selection import GridSearchCV
from dask_ml.linear_model import LogisticRegression
import nltk
nltk.download('stopwords')
nltk.download('vader_lexicon')
from nltk.corpus import stopwords
import re
import string
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
!pip3 install emoji
import emoji

depression_stress_words = {
    'anxious', 'lonely', 'hopeless', 'empty', 'helpless', 'worthless', 'tired', 'exhausted', 'fatigued', 'miserable',
    'hopelessness', 'numb', 'irritable', 'sorrow', 'suffering', 'suicidal', 'tearful', 'troubled', 'vulnerable', 'crisis',
    'despair', 'disheartened', 'dismal', 'downhearted', 'forlorn', 'gloomy', 'grief', 'melancholy', 'mournful', 'sad',
    'tragic', 'unhappy', 'dejected', 'demoralized', 'devastated', 'discouraged', 'downtrodden', 'low-spirited', 'blue',
    'heartbroken', 'hurt', 'injured', 'pained', 'distressed', 'tormented', 'angry', 'annoyed', 'frustrated', 'irritated',
    'fearful', 'afraid', 'scared', 'terrified', 'worried', 'concerned', 'anxiety', 'panic', 'stress', 'depression', 'suicide',
    'self-harm', 'overwhelmed', 'pressure', 'mental health', 'loneliness', 'isolation', 'disconnected', 'unloved', 'unworthy',
    'worthlessness', 'worthless', 'rejected', 'failure', 'inadequate', 'guilty', 'ashamed', 'self-blame', 'negativity', 'hopeless',
    'meaningless', 'numbness', 'apathy', 'loss', 'trapped', 'confused', 'disoriented', 'disillusioned', 'betrayed', 'humiliated',
    'empty', 'exhausted', 'fatigue', 'irritability', 'anger', 'rage', 'mood swings', 'insomnia', 'sleep problems', 'nightmares',
    'loss of interest', 'withdrawal', 'social withdrawal', 'lethargic', 'energy loss', 'pessimism', 'dysphoria', 'bad mood',
    'foggy', 'lethargy', 'inability to concentrate'
}


stress_scores = {'death': 5, 'emergency': 4, 'urgent': 4, 
                           'crisis': 4, 'trauma': 4, 'disaster': 4, 
                           'injury': 4, 'hospital': 4, 'catastrophe': 4, 
                           'danger': 3, 'threat': 3, 'violence': 3, 'crime': 3,
                           'attack': 3, 'fear': 3, 'panic': 3, 'anxiety': 3, 
                           'terror': 3, 'conflict': 2, 'pressure': 1, 
                           'deadline': 2, 'overwhelmed': 2, 'frustrated': 2, 
                           'exhausted': 2, 'tired': 2, 'fatigue': 2, 'stress': 2, 
                           'anxious': 2, 'challenging': 1, 'difficult': 1, 'complicated': 1,
                           'busy': 1, 'demanding': 1, 'responsibility': 1, 
                           'workload': 1, 'expectations': 1, 'change': 1,
                           'uncertainty': 1, 'unknown': 1, 'new': 1, 
                           'fired': 3, 'laid off': 3, 'termination': 3,
                           'budget cuts': 2, 'bankruptcy': 4, 'divorce': 4, 
                           'break-up': 3, 'death of a loved one': 4, 
                           'health issues': 3, 'major life changes': 2, 
                           'financial problems': 3, 'relationship issues': 2, 
                           'loneliness': 2, 'isolation': 2, 'rejection': 2, 
                           'failure': 2, 'disappointment': 1, 'homesick': 1, 
                           'missing someone': 1, 'nostalgia': 1, 'gratitude': -2, 
                           'joy': -2, 'happiness': -2, 'relief': -2, 'satisfaction': -2, 
                           'contentment': -2, 'calm': -2, 'peace': -2, 'tranquility': -2, 
                           'meditation': -2, 'mindfulness': -2, 'positivity': -2, 'optimism': -2,
                           'hope': -2, 'grateful': -1, 'happy': -1, 'excited': -1, 'adventurous': -1, 
                           'curious': -1, 'motivated': -1, 'enthusiastic': -1, 'confident': -1, 
                           'proud': -1, 'inspired': -1, 'anger': 3, 'rage': 4, 'irritation': 2,
                           'annoyance': 2, 'disgust': 3, 'envy': 2, 'jealousy': 2, 'guilt': 2, 
                           'stressed': 2, 'lonely': 1, 'hopeless': 5, 'empty': 1, 'helpless': 4, 
                           'worthless': 4, 'fatigued': 2, 'miserable': 3, 'hopelessness': 4, 
                           'numb': 3, 'irritable': 2, 'sorrow': 3, 'suffering': 4, 
                           'suicidal': 5, 'tearful': 3, 'troubled': 2, 'vulnerable': 2, 
                           'despair': 4, 'disheartened': 3, 'dismal': 3, 'downhearted': 3, 
                           'forlorn': 3, 'gloomy': 3, 'grief': 4, 'melancholy': 4, 'mournful': 4, 
                           'sad': 3, 'tragic': 5, 'unhappy': 2, 'dejected': 3, 'demoralized': 4, 
                           'devastated': 5, 'discouraged': 3, 'downtrodden': 4, 'low-spirited': 3, 
                           'blue': 3, 'heartbroken': 4, 'hurt': 2, 'injured': 2, 'pained': 2, 
                           'distressed': 3, 'tormented': 4, 'angry': 2, 'annoyed': 1, 'irritated': 2,
                           'fearful': 3, 'afraid': 3, 'scared': 4, 'terrified': 5, 'worried': 2, 
                           'concerned': 2, 'depression': 5, 'suicide': 5, 'self-harm': 4, 
                           'mental health': 4, 'disconnected': 2, 'unloved': 3, 'unworthy': 3, 
                           'worthlessness': 4, 'rejected': 3, 'inadequate': 3, 'guilty': 3, 
                           'ashamed': 3, 'self-blame': 4, 'negativity': 3, 'meaningless': 3, 
                           'numbness': 3, 'apathy': 1, 'loss': 3, 'trapped': 4, 'confused': 2, 
                           'disoriented': 2, 'disillusioned': 2, 'betrayed': 3, 'humiliated': 3, 
                           'irritability': 2, 'mood swings': 2, 'insomnia': 3, 'sleep problems': 2, 
                           'nightmares': 3, 'loss of interest': 2, 'withdrawal': 2, 'social withdrawal': 2,
                           'lethargic': 2, 'energy loss': 2, 'pessimism': 2, 'dysphoria': 3, 'bad mood': 1,
                           'foggy': 2, 'lethargy': 2, 'inability to concentrate': 2}


stress_emojis={
    "\U0001F600": -2, # grinning face
    "\U0001F603": -2, # grinning face with big eyes
    "\U0001F604": -2, # grinning face with smiling eyes
    "\U0001F601": -2, # beaming face with smiling eyes
    "\U0001F606": -2, # grinning squinting face
    "\U0001F605": 1, # grinning face with sweat
    "\U0001F923": -1, # rolling on the floor crying
    "\U0001F602": -1, # face with tears of joy
    "\U0001F642": -2, # slightly smiling face
    "\U0001F643": 3, # upside-down face
    "\U0001FAE0": 1, # meltng face
    "\U0001F609": -2, # winking face
    "\U0001F60A": -2, # smiling face with smiling eyes
    "\U0001F607": -2, # smiling face with halo
    "\U0001F970": -2, # smiling face with hearts
    "\U0001F60D": -2, # smiling face with heart eyes
    "\U0001F929": -2, # star-struck
    "\U0001F618": -2, # face blowing a kiss
    "\U0001F617": -2, # kissing face
    "\U0001F61A": -2, # kissing face with eyes closed
    "\U0001F619": -2, # kissing face with smiling eyes
    "\U0001F972": 2, # smiling face with tear
    "\U0001F60B": -2, # face savoring food
    "\U0001F61B": -2, # face with tongue
    "\U0001F61C": -1, # winking face with tongue
    "\U0001F929": -1, # zany face
    "\U0001F61D": -1, # squinting face with tongue
    "\U0001F911": -2, # money-mouth face
    "\U0001F917": -1, # smiling face with hands
    "\U0001F92D": -1, # face with hand over mouth
    "\U0001FAE2": 2, # face with opne eyes and hand over mouth
    "\U0001FAE3": 3, # face with peeking eye
    "\U0001F92B": 1, # shushing face
    "\U0001F914": 2, # thinking face
    "\U0001FAE1": -1, # saltuing face
    "\U0001F910": 1, # zipper-mouth face
    "\U0001F928": 2, #face with raising eyebrow
    "\U0001F610": 2, # neutral face
    "\U0001F611": 1, # expresionless face
    "\U0001F636": 1, # face without mouth
    "\U0001FAE5": 1, # dotted line face
    "\U0001F636": -1, # face in the clouds
    "\U0001F60F": -2, # smirk
    "\U0001F612": 2, # unamused face
    "\U0001F644": 3, # face with rolling eyes
    "\U0001F62C": 2, # grimacing face
    "\U0001F62E": 2, # face exhaling
    "\U0001F925": 1, # lying face
    "\U0001F60C": -2, # relieved face
    "\U0001F614": 5, # pensive face
    "\U0001F62A": 4, # sleepy face
    "\U0001F924": -2, # drooling face
    "\U0001F634": 1, # sleeping face
    "\U0001F637": -2, # face with medical mask 
    "\U0001F912": -2, # face with thermometer
    "\U0001F915": -1, # face with bandage
    "\U0001F922": -1, # nauseated face
    "\U0001F92E": -1, # face vomiting
    "\U0001F927": -2, # sneezing face
    "\U0001F975": -2, # hot face
    "\U0001F976": -2, # cold face
    "\U0001F974": 1, # woozy face
    "\U0001F635": -1, # face with crossed-out-eyes
    "\U0001F92F": 2, # exploding head
    "\U0001F60E": -2, # smiling face with sunglasses
    "\U0001F913": -2, # nerd face
    "\U0001F615": 1, # confused face
    "\U0001FAE4": 2, # face with diagonal mouth
    "\U0001F61F": 3, # worried face
    "\U0001F641": 3, # slightly frowning face
    "\U0001F62E": 1, # face with open mouth
    "\U0001F62F": 1, # hushed face 
    "\U0001F633": 2, # flushed face
    "\U0001F97A": -1, # pleading face
    "\U0001F979": 2, # face holding back tears
    "\U0001F626": 2, # frowning face with open mouth
    "\U0001F627": 2, # anguished face
    "\U0001F62B": 2, # fearful face
    "\U0001F630": 5, # anxious face with sweat
    "\U0001F625": 5, # sad but relieved face
    "\U0001F622": 4, # crying face
    "\U0001F62D": 4, # loudly crying face
    "\U0001F631": 3, # face screaming in fear
    "\U0001F616": 3, # confused face
    "\U0001F623": 5, # perservering face
    "\U0001F61E": 5, # dissapointed face
    "\U0001F613": 5, # downcast face with sweat
    "\U0001F629": 3, # weary face
    "\U0001F62B": 3, # tired face
    "\U0001F971": 2, # yawnging face
    "\U0001F624": 2, # face with steam from nose
    "\U0001F621": 1, # enraged face
    "\U0001F620": 1, # angry face
    "\U0001F92C": 3, # face with symbols on mouth
    "\U0001F63F": 4, # crying cat
    "\U0001F63E": 4, # pouting cat
    "\U0001F494": 4, # broken heart
    "\U0001F44D": -2, # thumbs up
    "\U0001F44E": 3, # thumbs down

}

stop_words = set(stopwords.words('english'))
url_pattern = re.compile(r'https?://\S+|www\.\S+')
stop_pattern = re.compile(r'\b(' + '|'.join(stop_words) + r')\b\s*')
mention_pattern = re.compile(r'@\w+')
hashtag_pattern = re.compile(r'#\w+')
emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   "]+", flags=re.UNICODE)
def calculate_score(text):
    score = 0
    words = text.lower().split()
    for word in words:
        if word in stress_scores:
            score += stress_scores[word]
    # Keep score between -5 and 5
    if score > 5:
        score = 5
    elif score < -5:
        score = -5
    return score


# emoji score 
def emoji_score(text):
    score = 0
    words = text.lower().split()
    for char in words:
        if char in stress_emojis:
            score += stress_emojis[char]
    if score > 5:
      score = 5
    elif score < -2:
        score = -2
    return score


def custom_tokenize(text):
    try:
      lang = detect(text)
    except:
      lang = "error"
      print("This tweet throws an error:", text)
    
    if lang == "en":
    # Remove URLs
        text = url_pattern.sub('', str(text))
        # Remove mentions
        text = mention_pattern.sub('', str(text))
        # Remove hashtags, but keep the words
        text = hashtag_pattern.sub('', str(text))
        # Convert to lowercase
        text = text.lower()
        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))
        # Tokenize tweet into words
        words = text.split()
        # Remove stop words
        words = [word for word in words if word not in stop_words]
        return " ".join(words)
    else:
        return ""

# chunk_size = 50000
# clean_tweets = []
#outputf = "stress_depression_aylene1.csv"
analyzer = SentimentIntensityAnalyzer()

# Read the CSV file using Dask
df = dd.read_csv('/content/drive/MyDrive/Colab Notebooks/stress_depression9_aylene.csv', usecols=["Text", "Username"], low_memory=False, lineterminator='\n')

# Drop rows with at least one missing value
df = df.map_partitions(lambda partition: partition.dropna(how='any'))
# df = df.map_partitions(lambda df: df.dropna(subset=['Text',"Username"]), meta=df)
# Filter out rows that don't contain any depression words
df = df[df['Text'].str.contains('|'.join(depression_stress_words))]
# Tokenize tweets into words
df['Text'] = df['Text'].apply(custom_tokenize, meta=('Text', 'object'))
# Add a new column to the dataframe with the depression/stress score
df['depression_stress_score'] = df['Text'].apply(calculate_score, meta=('x', 'f8'))

# Calculate VADER sentiment for each partition of the dataframe
df['vader_sentiment'] = df.map_partitions(lambda part: part['Text'].apply(lambda x: analyzer.polarity_scores(x)['compound']))
df['emojie_stress_score'] = df['Text'].apply(emoji_score, meta=('x', 'f8'))
# Write cleaned data to CSV file
df.to_csv("stress_depression9_aylene9.csv", single_file=True, index=False)
print(df.head())


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.7/148.7 kB 3.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993243 sha256=36b5bd40a2d53693bccfd8e873a41bae5e3fbb713321660be11f4c20f82df065
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=ab7cd6470b2e5ed36f8a5948f764cfd36a8ae8688a0413675c35469d3d9a5573
  Stored in directory: /root/.cache/pip/wheels/02/3d/88/51a592b9ad17e7899126563698b4e3961983ebe85747228ba6
Successfully built emoji
                                                Text         Username  \
1  glad malia👍❣sadly seems media main culprits co...     Kate18660449   
2  lot shit happen await sky pirates info lt3 tan...     vanetheworld   
4  yep pete dropping raises likelihood viability ...      MaxKennerly   
5  using land farming timber natural resources ma...   killinger_lexi   
6  📍seminar mexico qampa thread olympics people s...  MedvedevaNation   

   depression_stress_score  vader_sentiment  emojie_stress_

In [ ]:
import pandas as pd

# read the CSV file into a pandas DataFrame
df = pd.read_csv('stress_depression9_aylene9.csv')

# drop rows where the 'text' column is empty
df.dropna(subset=['Text'], inplace=True)

# write the updated DataFrame back to a new CSV file
df.to_csv('stress_depression9_aylene9-1.csv', index=False)